In [4]:
import time
import cv2
import numpy as np
from collections import deque
from mss import mss
from ultralytics import YOLO
import keyboard
import win32api
import ctypes
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
'''
IN A NUTSHELL
CONCEPT OF THE SCRIPT:
1. Capture the screen indefinitely 800 x 600 px, if ur monitor is larger than that please change ur resolution first.
2. Use YOLOv11 to detect any object on each frame.
3. Use Deep SORT to track detected objects across frames.

TRACK LOGIC:
- If class 1 (E.g, bomb) is detected, prioritize a selective and safe tracking on class 0 (E.g, fruit) in order to minimize risk of collateral. In which, the farthest fruit from the bomb will be tracked and targeted first.
- If class 1 is not detected, track all detected class 0. And go crazy

USAGE:
Press 's' to start/pause the script.
Press 'q' to quit the script.
'''

In [416]:
# Load YOLO
model = YOLO("bestModel.pt")
model.to("cuda")

# Initialize Deep SORT tracker
tracker = DeepSort(
    max_age=2,  # Balanced for retention and speed
    n_init=1,
    nms_max_overlap=1.0,
    embedder="mobilenet",
    half=True,
)

In [417]:
model.info()

YOLO11s summary: 181 layers, 9,428,566 parameters, 0 gradients, 21.6 GFLOPs


(181, 9428566, 0, 21.550643200000003)

In [6]:
# GLOBAL VARIABLES, might as well just put it on another file later.
MON = {"top": 0, "left": 0, "width": 1024, "height": 768}   
MODEL_INPUT = 416  # Reduced for faster inference
SCREEN_W = ctypes.windll.user32.GetSystemMetrics(0)
SCREEN_H = ctypes.windll.user32.GetSystemMetrics(1)
YOLO_CONF = 0.15  # Increased to reduce low-confidence detections
IOU_SKIP_THRESH = 0.2
SAFE_NEW_IOU = 0.1
FORCE_DROP_IOU = 0.5
MIN_CENTER_DISTANCE = 30.0
MAX_CURSOR_DISTANCE = 800.0
MIN_MOVEMENT_THRESHOLD = 10.0

In [61]:
def letterbox_image(img, new_size=MODEL_INPUT, color=(114,114,114)):
    h, w = img.shape[:2]
    scale = min(new_size / w, new_size / h)
    nw, nh = int(w * scale), int(h * scale)
    img_resized = cv2.resize(img, (nw, nh))
    pad_x = (new_size - nw) // 2
    pad_y = (new_size - nh) // 2
    top, bottom = pad_y, new_size - nh - pad_y
    left, right = pad_x, new_size - nw - pad_x
    img_padded = cv2.copyMakeBorder(img_resized, top, bottom, left, right,
                                    cv2.BORDER_CONSTANT, value=color)
    return img_padded, scale, pad_x, pad_y

def unletterbox_box(x1, y1, x2, y2, scale, pad_x, pad_y):
    x1_u = (x1 - pad_x) / scale
    y1_u = (y1 - pad_y) / scale
    x2_u = (x2 - pad_x) / scale
    y2_u = (y2 - pad_y) / scale
    return x1_u, y1_u, x2_u, y2_u

def iou_box(a, b):
    x1, y1, x2, y2 = a
    ox1, oy1, ox2, oy2 = b
    inter_x1 = max(x1, ox1)
    inter_y1 = max(y1, oy1)
    inter_x2 = min(x2, ox2)
    inter_y2 = min(y2, oy2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area1 = max(0, x2 - x1) * max(0, y2 - y1)
    area2 = max(0, ox2 - ox1) * max(0, oy2 - oy1)
    union = area1 + area2 - inter_area
    return inter_area / union if union > 0 else 0.0

def clamp_box_to_image(box, img_w, img_h):
    x1, y1, x2, y2 = box
    x1 = max(0, min(img_w - 1, x1))
    x2 = max(0, min(img_w - 1, x2))
    y1 = max(0, min(img_h - 1, y1))
    y2 = max(0, min(img_h - 1, y2))
    if x2 <= x1 or y2 <= y1:
        return None
    return (x1, y1, x2, y2)

def center_distance(box1, box2):
    x1, y1, x2, y2 = box1
    ox1, oy1, ox2, oy2 = box2
    cx1, cy1 = (x1 + x2) / 2.0, (y1 + y2) / 2.0
    cx2, cy2 = (ox1 + ox2) / 2.0, (oy1 + oy2) / 2.0
    return np.sqrt((cx1 - cx2) ** 2 + (cy1 - cy2) ** 2)

def point_distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def is_path_safe(current_pos, target_pos, class1_boxes, margin=20.0, max_distance=MAX_CURSOR_DISTANCE):
    cx, cy = current_pos
    tx, ty = target_pos
    distance = point_distance((cx, cy), (tx, ty))
    if distance > max_distance:
        print(f"Path rejected: Distance {distance:.2f} exceeds max {max_distance}")
        return False
    for x1, y1, x2, y2 in class1_boxes:
        x1_m, y1_m, x2_m, y2_m = x1 - margin, y1 - margin, x2 + margin, y2 + margin
        if x1_m <= tx <= x2_m and y1_m <= ty <= y2_m:
            return False
        dx, dy = tx - cx, ty - cy
        # Fixed 3 steps for path checking
        for t in np.linspace(0, 1, 3):
            px = cx + t * dx
            py = cy + t * dy
            if x1_m <= px <= x2_m and y1_m <= py <= y2_m:
                return False
    return True

In [390]:
timer = deque()
with mss() as sct:
    for i in range(200):
        st = time.time()
        a = np.asarray(sct.grab(MON))[:, :, :3]
        img_padded, scale, pad_x, pad_y = letterbox_image(a, new_size=MODEL_INPUT)
        results = model(img_padded, stream=False, conf=YOLO_CONF, iou=0.2, int8=True, verbose=False)
        et = time.time()
        time_taken = et - st
        timer.append(time_taken)

    print(f"Average inference time over 200 runs: {np.mean(timer)*1000:.2f} ms")

Average inference time over 200 runs: 36.73 ms


In [391]:
timer = deque()
with mss() as sct:
    for i in range(200):
        st = time.time()
        a = np.asarray(sct.grab(MON))[:, :, :3]
        results = model(a, stream=False, conf=YOLO_CONF, iou=0.2, int8=True, verbose=False)
        et = time.time()
        time_taken = et - st
        timer.append(time_taken)

    print(f"Average inference time over 200 runs: {np.mean(timer)*1000:.2f} ms")

Average inference time over 200 runs: 36.55 ms


In [392]:
timer = deque()
with mss() as sct:
    for i in range(200):
        st = time.time()
        a = np.asarray(sct.grab(MON))[:, :, :3]
        results = model(a, stream=True, conf=YOLO_CONF, iou=0.2, verbose=False, int8=True)
        et = time.time()
        boxes = [i.boxes for i in results]
        time_taken = et - st
        timer.append(time_taken)

    print(f"Average inference time over 200 runs: {np.mean(timer)*1000:.2f} ms")

Average inference time over 200 runs: 11.84 ms


In [393]:
timer = deque()
with mss() as sct:
    for i in range(200):
        st = time.time()
        a = np.asarray(sct.grab(MON))[:, :, :3]
        results = model(a, stream=True, conf=YOLO_CONF, iou=0.2, verbose=False, half=True)
        et = time.time()
        boxes = [i.boxes for i in results]
        time_taken = et - st
        timer.append(time_taken)

    print(f"Average inference time over 200 runs: {np.mean(timer)*1000:.2f} ms")

Average inference time over 200 runs: 12.07 ms


In [394]:
model.export(format='engine', half=True)

WARNING TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.197  Python-3.13.9 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)

PyTorch: starting from 'best11s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.2 MB)

ONNX: starting export with onnx 1.19.0 opset 19...
ONNX: slimming with onnxslim 0.1.68...
ONNX: export success  2.5s, saved as 'best11s.onnx' (10.1 MB)

TensorRT: starting export with TensorRT 10.13.3.9...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 6, 8400) DataType.FLOAT
TensorRT: building FP16 engine as best11s.engine
TensorRT: export success  252.4s, saved as 'best11s.engine' (8.8 MB)

Export complete (252.5s)
Results saved to D:\Project-personal\fruitDet_CV\SRC
Predict:         yolo predict task=detect model=best11s.engine imgsz=640 half 
Validate:        yolo val task=detect model=best11s.engine imgsz=640 data=../DATA/Finalized/data.yam

'best11s.engine'

In [401]:
model = YOLO("best11s.engine", task='detect')


timer = deque()
with mss() as sct:
    for i in range(200):
        st = time.time()
        a = np.asarray(sct.grab(MON))[:, :, :3]
        results = model(a, stream=True, conf=YOLO_CONF, iou=0.2, verbose=False, half=True)
        et = time.time()
        boxes = [i.boxes for i in results]
        time_taken = et - st
        timer.append(time_taken)

    print(f"Average inference time over 200 runs: {np.mean(timer)*1000:.2f} ms")

Loading best11s.engine for TensorRT inference...
Average inference time over 200 runs: 13.09 ms


In [402]:
model = YOLO("best11s.engine", task='detect')


timer = deque()
with mss() as sct:
    for i in range(200):
        st = time.time()
        a = np.asarray(sct.grab(MON))[:, :, :3]
        results = model(a, stream=True, conf=YOLO_CONF, iou=0.2, verbose=False, half=True)
        et = time.time()
        boxes = [i.boxes for i in results]
        time_taken = et - st
        timer.append(time_taken)

    print(f"Average inference time over 200 runs: {np.mean(timer)*1000:.2f} ms")

Loading best11s.engine for TensorRT inference...
Average inference time over 200 runs: 13.48 ms


In [408]:
import time
import numpy as np
import dxcam
from collections import deque
from ultralytics import YOLO

# Load the TensorRT engine
model = YOLO("best11s.engine", task='detect')

# Initialize DXCam (Target specific monitor if needed, e.g., output_idx=0)
# 'output_color="BGR"' matches YOLO requirements perfectly, skipping color conversion
camera = dxcam.create(output_idx=0, output_color="BGR")

# Define capture region (Left, Top, Right, Bottom)
# Adjust these values to match your 'MON' variable
region = (0, 0, 640, 640) 

# Warmup: Run inference once to load engine into VRAM
print("Warming up...")
model(np.zeros((640, 640, 3), dtype=np.uint8), verbose=False)

timer = deque()

# Start capturing
print("Starting benchmark...")
camera.start(target_fps=144, region=region) # continuous capture is faster than grab()

for i in range(2000):
    st = time.time()
    
    # Get the latest frame from the buffer (Non-blocking usually)
    image = camera.get_latest_frame() 
    
    # Loop until a new frame is actually available (prevents inferring same frame twice)
    # In a real aimbot/bot loop, you might skip this to run as fast as possible, 
    # but for benchmarking, we want fresh frames.
    while image is None:
        image = camera.get_latest_frame()

    # Inference
    # Removed stream=True: unnecessary overhead for single-image loops
    results = model(image, conf=0.5, iou=0.2, verbose=False, half=True, stream=True)
    
    et = time.time()
    
    # Access boxes to ensure inference is actually complete before stopping timer
    boxes = [i.boxes for i in results]
    
    timer.append(et - st)

camera.stop()
print(f"Average inference time over 2000 runs: {np.mean(timer)*1000:.2f} ms")

You already created a DXCamera Instance for Device 0--Output 0!
Returning the existed instance...
To change capture parameters you can manually delete the old object using `del obj`.
Warming up...
Loading best11s.engine for TensorRT inference...
Starting benchmark...
Screen Capture FPS: 101
Average inference time over 2000 runs: 1.97 ms


In [ ]:
auto_aim = False
target_lock = None
mouse_pressed = False
img_h, img_w = MON["height"], MON["width"]

def startAutoSlicer(visualize=True):
    global auto_aim, target_lock, mouse_pressed

    fps_deque = deque(maxlen=60)
    frame_count = 0

    # Start screen capture
    with mss() as sct:
        while True:

            # Capture screen 
            t0 = time.time()
            frame_count += 1

            # Check keyboard input every 5 frames
            if frame_count % 5 == 0 and keyboard.is_pressed("e"):
                auto_aim = not auto_aim

            img = np.asarray(sct.grab(MON))[:, :, :3]   # Capture only RGB channels

            img_resized, scale, pad_x, pad_y = letterbox_image(img, new_size=MODEL_INPUT)

            # YOLO inference
            results = model(img_resized, stream=False, conf=YOLO_CONF, iou=0.2, int8=True, verbose=False)

            detections = []
            class0_dets = []
            class1_boxes = []  # Directly store confirmed class 1 boxes
            for box in results[0].boxes:
                cls = int(box.cls[0].cpu())
                if cls not in (0, 1):
                    continue
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(float)
                ux1, uy1, ux2, uy2 = unletterbox_box(x1, y1, x2, y2, scale, pad_x, pad_y)
                clamped = clamp_box_to_image((ux1, uy1, ux2, uy2), img_w, img_h)
                if clamped is None:
                    continue
                ux1, uy1, ux2, uy2 = clamped
                w = ux2 - ux1
                h = uy2 - uy1
                conf = float(box.conf[0].cpu())
                if cls == 0:
                    detections.append(([int(ux1), int(uy1), int(w), int(h)], conf, cls))
                    class0_dets.append((ux1, uy1, ux2, uy2))
                else:
                    class1_boxes.append((ux1, uy1, ux2, uy2))  # Treat class 1 detections as confirmed boxes

            print(f"Class 0 dets: {len(class0_dets)}, Class 1 dets: {len(class1_boxes)}")

            # DeepSort tracking for class 0 only
            tracks = tracker.update_tracks(detections, frame=img)
            class0_tracks = [t for t in tracks if t.det_class == 0 and t.is_confirmed()]
            print(f"Confirmed class 0 tracks: {len(class0_tracks)}")

            target_center = None
            chosen_track = None
            path_to_draw = None
            is_fallback = False

            current_cursor_pos = win32api.GetCursorPos()
            current_cursor_pos = (current_cursor_pos[0] - MON["left"], current_cursor_pos[1] - MON["top"])

            # Single-pass target selection
            best_track = None
            best_distance = float('inf')
            last_track_id = getattr(startAutoSlicer, "last_track_id", None)

            for t in class0_tracks:
                x1, y1, x2, y2 = t.to_ltrb()
                cx, cy = (x1 + x2) / 2.0, (y1 + y2) / 2.0
                cursor_distance = point_distance(current_cursor_pos, (cx, cy))

                # Prefer current target if safe
                if t.track_id == last_track_id and class1_boxes:
                    overlap = max([iou_box((x1, y1, x2, y2), b) for b in class1_boxes], default=0.0)
                    min_dist = min([center_distance((x1, y1, x2, y2), b) for b in class1_boxes], default=float('inf'))
                    if overlap > FORCE_DROP_IOU or min_dist < MIN_CENTER_DISTANCE or not is_path_safe(current_cursor_pos, (cx, cy), class1_boxes):
                        continue
                    best_track = t
                    best_distance = cursor_distance
                    break
                elif cursor_distance < best_distance:
                    if not class1_boxes or (
                        max([iou_box((x1, y1, x2, y2), b) for b in class1_boxes], default=0.0) <= SAFE_NEW_IOU and
                        min([center_distance((x1, y1, x2, y2), b) for b in class1_boxes], default=float('inf')) >= MIN_CENTER_DISTANCE and
                        is_path_safe(current_cursor_pos, (cx, cy), class1_boxes)
                    ):
                        best_track = t
                        best_distance = cursor_distance

            if best_track is not None:
                x1, y1, x2, y2 = best_track.to_ltrb()
                target_center = ((x1 + x2) / 2.0, (y1 + y2) / 2.0)
                chosen_track = best_track
                print(f"Selected track {best_track.track_id} with cursor distance: {best_distance:.2f}")

            # Aim prediction + move + click hold
            if auto_aim and target_center is not None:
                now = time.time()
                if hasattr(startAutoSlicer, "last_center") and startAutoSlicer.last_center is not None and startAutoSlicer.last_time:
                    dt = now - startAutoSlicer.last_time
                    if dt > 0:
                        vx = (target_center[0] - startAutoSlicer.last_center[0]) / dt
                        vy = (target_center[1] - startAutoSlicer.last_center[1]) / dt
                        latency = 0.08
                        pred_x = target_center[0] + vx * latency
                        pred_y = target_center[1] + vy * latency
                    else:
                        pred_x, pred_y = target_center
                else:
                    pred_x, pred_y = target_center

                if point_distance((pred_x, pred_y), current_cursor_pos) < MIN_MOVEMENT_THRESHOLD:
                    print(f"Skipping move to track {chosen_track.track_id}: movement too small")
                    path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                else:
                    startAutoSlicer.last_center = target_center
                    startAutoSlicer.last_time = now
                    mapped_x = int(MON["left"] + pred_x)
                    mapped_y = int(MON["top"] + pred_y)
                    mapped_x = max(0, min(SCREEN_W - 1, mapped_x))
                    mapped_y = max(0, min(SCREEN_H - 1, mapped_y))
                    target_lock = (mapped_x, mapped_y)

                    path_safe = not class1_boxes or is_path_safe(current_cursor_pos, (pred_x, pred_y), class1_boxes)
                    if chosen_track.track_id != last_track_id:
                        if mouse_pressed:
                            win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                            mouse_pressed = False
                            time.sleep(0.005)  # Reduced delay
                        if path_safe:
                            win32api.SetCursorPos(target_lock)
                            win32api.mouse_event(0x0002, 0, 0, 0, 0)  # Press
                            mouse_pressed = True
                            startAutoSlicer.last_track_id = chosen_track.track_id
                            path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                            print(f"Mouse moved to {target_lock}, clicked")
                        else:
                            print(f"Skipping move to track {chosen_track.track_id}: unsafe path")
                            target_lock = None
                            startAutoSlicer.last_track_id = None
                    else:
                        if path_safe:
                            win32api.SetCursorPos(target_lock)
                            if not mouse_pressed:
                                win32api.mouse_event(0x0002, 0, 0, 0, 0)  # Press
                                mouse_pressed = True
                            path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                            print(f"Mouse moved to {target_lock}, clicked (same track)")
                        else:
                            if mouse_pressed:
                                win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                                mouse_pressed = False
                            print(f"Releasing mouse: unsafe path for track {chosen_track.track_id}")
                            target_lock = None
                            startAutoSlicer.last_track_id = None
            else:
                if mouse_pressed:
                    win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                    mouse_pressed = False
                    time.sleep(0.005)
                target_lock = None
                startAutoSlicer.last_center = None
                startAutoSlicer.last_time = None
                startAutoSlicer.last_track_id = None
                print("Releasing mouse: no target or auto_aim off")

            elapsed = time.time() - t0
            fps = 1.0 / elapsed if elapsed > 0 else 0.0
            fps_deque.append(fps)
            print(f"Iteration FPS: {fps:.2f}")

            if visualize:
                frame = img.copy()
                cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                cv2.putText(frame, f"Aim: {'ON' if auto_aim else 'OFF'}", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

                # Draw only selected track and class 1 boxes
                if chosen_track is not None:
                    x1, y1, x2, y2 = map(int, chosen_track.to_ltrb())
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID {chosen_track.track_id} C0", (x1, max(0, y1-6)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                for x1, y1, x2, y2 in class1_boxes:
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

                if target_center is not None:
                    cv2.circle(frame, (int(target_center[0]), int(target_center[1])), 4, (0, 255, 255), -1)
                if target_lock is not None:
                    vis_pred_x = target_lock[0] - MON["left"]
                    vis_pred_y = target_lock[1] - MON["top"]
                    cv2.circle(frame, (int(vis_pred_x), int(vis_pred_y)), 4, (255, 0, 255), -1)
                if path_to_draw is not None:
                    start, end = path_to_draw
                    cv2.line(frame, (int(start[0]), int(start[1])), (int(end[0]), int(end[1])),
                             (255, 165, 0), 1)

                cv2.imshow("screenshot", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    cv2.destroyAllWindows()
                    break

### Normal shit

In [ ]:
import time
import cv2
import numpy as np
from collections import deque
from mss import mss
from ultralytics import YOLO
import keyboard
import win32api
import ctypes
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
try:
    ctypes.windll.user32.SetProcessDPIAware()
except Exception:
    pass

# Load YOLO
model = YOLO("best11s.pt")
model.to("cuda")

# DeepSORT init
tracker = DeepSort(
    max_age=2,  # Balanced for retention and speed
    n_init=1,
    nms_max_overlap=1.0,
    embedder="mobilenet",
    half=True,
)

auto_aim = False
target_lock = None

# Precompute constants
MON = {"top": 0, "left": 0, "width": 1024, "height": 768}
MODEL_INPUT = 416  # Reduced for faster inference
SCREEN_W = ctypes.windll.user32.GetSystemMetrics(0)
SCREEN_H = ctypes.windll.user32.GetSystemMetrics(1)
YOLO_CONF = 0.15  # Increased to reduce low-confidence detections
IOU_SKIP_THRESH = 0.2
SAFE_NEW_IOU = 0.1
FORCE_DROP_IOU = 0.5
MIN_CENTER_DISTANCE = 30.0
MAX_CURSOR_DISTANCE = 800.0
MIN_MOVEMENT_THRESHOLD = 10.0

def letterbox_image(img, new_size=MODEL_INPUT, color=(114,114,114)):
    h, w = img.shape[:2]
    scale = min(new_size / w, new_size / h)
    nw, nh = int(w * scale), int(h * scale)
    img_resized = cv2.resize(img, (nw, nh))
    pad_x = (new_size - nw) // 2
    pad_y = (new_size - nh) // 2
    top, bottom = pad_y, new_size - nh - pad_y
    left, right = pad_x, new_size - nw - pad_x
    img_padded = cv2.copyMakeBorder(img_resized, top, bottom, left, right,
                                    cv2.BORDER_CONSTANT, value=color)
    return img_padded, scale, pad_x, pad_y

def unletterbox_box(x1, y1, x2, y2, scale, pad_x, pad_y):
    x1_u = (x1 - pad_x) / scale
    y1_u = (y1 - pad_y) / scale
    x2_u = (x2 - pad_x) / scale
    y2_u = (y2 - pad_y) / scale
    return x1_u, y1_u, x2_u, y2_u

def iou_box(a, b):
    x1, y1, x2, y2 = a
    ox1, oy1, ox2, oy2 = b
    inter_x1 = max(x1, ox1)
    inter_y1 = max(y1, oy1)
    inter_x2 = min(x2, ox2)
    inter_y2 = min(y2, oy2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area1 = max(0, x2 - x1) * max(0, y2 - y1)
    area2 = max(0, ox2 - ox1) * max(0, oy2 - oy1)
    union = area1 + area2 - inter_area
    return inter_area / union if union > 0 else 0.0

def clamp_box_to_image(box, img_w, img_h):
    x1, y1, x2, y2 = box
    x1 = max(0, min(img_w - 1, x1))
    x2 = max(0, min(img_w - 1, x2))
    y1 = max(0, min(img_h - 1, y1))
    y2 = max(0, min(img_h - 1, y2))
    if x2 <= x1 or y2 <= y1:
        return None
    return (x1, y1, x2, y2)

def center_distance(box1, box2):
    x1, y1, x2, y2 = box1
    ox1, oy1, ox2, oy2 = box2
    cx1, cy1 = (x1 + x2) / 2.0, (y1 + y2) / 2.0
    cx2, cy2 = (ox1 + ox2) / 2.0, (oy1 + oy2) / 2.0
    return np.sqrt((cx1 - cx2) ** 2 + (cy1 - cy2) ** 2)

def point_distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def is_path_safe(current_pos, target_pos, class1_boxes, margin=20.0, max_distance=MAX_CURSOR_DISTANCE):
    cx, cy = current_pos
    tx, ty = target_pos
    distance = point_distance((cx, cy), (tx, ty))
    if distance > max_distance:
        print(f"Path rejected: Distance {distance:.2f} exceeds max {max_distance}")
        return False
    for x1, y1, x2, y2 in class1_boxes:
        x1_m, y1_m, x2_m, y2_m = x1 - margin, y1 - margin, x2 + margin, y2 + margin
        if x1_m <= tx <= x2_m and y1_m <= ty <= y2_m:
            return False
        dx, dy = tx - cx, ty - cy
        # Fixed 3 steps for path checking
        for t in np.linspace(0, 1, 3):
            px = cx + t * dx
            py = cy + t * dy
            if x1_m <= px <= x2_m and y1_m <= py <= y2_m:
                return False
    return True

def startAutoSlicer(visualize=True):
    global auto_aim, target_lock

    fps_deque = deque(maxlen=60)
    frame_count = 0
    mouse_pressed = False  # Track mouse button state

    with mss() as sct:
        while True:
            t0 = time.time()
            frame_count += 1

            # Check keyboard input every 5 frames
            if frame_count % 5 == 0 and keyboard.is_pressed("e"):
                auto_aim = not auto_aim
                print(f"Auto Aim {'ON' if auto_aim else 'OFF'}")
                time.sleep(0.005)  # Reduced delay

            img = np.asarray(sct.grab(MON))[:, :, :3]
            img_h, img_w = img.shape[:2]

            img_resized, scale, pad_x, pad_y = letterbox_image(img, new_size=MODEL_INPUT)

            # YOLO inference
            results = model(img_resized, stream=False, conf=YOLO_CONF, iou=0.2, int8=True, verbose=False)

            detections = []
            class0_dets = []
            class1_boxes = []  # Directly store confirmed class 1 boxes
            for box in results[0].boxes:
                cls = int(box.cls[0].cpu())
                if cls not in (0, 1):
                    continue
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(float)
                ux1, uy1, ux2, uy2 = unletterbox_box(x1, y1, x2, y2, scale, pad_x, pad_y)
                clamped = clamp_box_to_image((ux1, uy1, ux2, uy2), img_w, img_h)
                if clamped is None:
                    continue
                ux1, uy1, ux2, uy2 = clamped
                w = ux2 - ux1
                h = uy2 - uy1
                conf = float(box.conf[0].cpu())
                if cls == 0:
                    detections.append(([int(ux1), int(uy1), int(w), int(h)], conf, cls))
                    class0_dets.append((ux1, uy1, ux2, uy2))
                else:
                    class1_boxes.append((ux1, uy1, ux2, uy2))  # Treat class 1 detections as confirmed boxes

            print(f"Class 0 dets: {len(class0_dets)}, Class 1 dets: {len(class1_boxes)}")

            # DeepSort tracking for class 0 only
            tracks = tracker.update_tracks(detections, frame=img)
            class0_tracks = [t for t in tracks if t.det_class == 0 and t.is_confirmed()]
            print(f"Confirmed class 0 tracks: {len(class0_tracks)}")

            target_center = None
            chosen_track = None
            path_to_draw = None
            is_fallback = False

            current_cursor_pos = win32api.GetCursorPos()
            current_cursor_pos = (current_cursor_pos[0] - MON["left"], current_cursor_pos[1] - MON["top"])

            # Single-pass target selection
            best_track = None
            best_distance = float('inf')
            last_track_id = getattr(startAutoSlicer, "last_track_id", None)

            for t in class0_tracks:
                x1, y1, x2, y2 = t.to_ltrb()
                cx, cy = (x1 + x2) / 2.0, (y1 + y2) / 2.0
                cursor_distance = point_distance(current_cursor_pos, (cx, cy))

                # Prefer current target if safe
                if t.track_id == last_track_id and class1_boxes:
                    overlap = max([iou_box((x1, y1, x2, y2), b) for b in class1_boxes], default=0.0)
                    min_dist = min([center_distance((x1, y1, x2, y2), b) for b in class1_boxes], default=float('inf'))
                    if overlap > FORCE_DROP_IOU or min_dist < MIN_CENTER_DISTANCE or not is_path_safe(current_cursor_pos, (cx, cy), class1_boxes):
                        continue
                    best_track = t
                    best_distance = cursor_distance
                    break
                elif cursor_distance < best_distance:
                    if not class1_boxes or (
                        max([iou_box((x1, y1, x2, y2), b) for b in class1_boxes], default=0.0) <= SAFE_NEW_IOU and
                        min([center_distance((x1, y1, x2, y2), b) for b in class1_boxes], default=float('inf')) >= MIN_CENTER_DISTANCE and
                        is_path_safe(current_cursor_pos, (cx, cy), class1_boxes)
                    ):
                        best_track = t
                        best_distance = cursor_distance

            if best_track is not None:
                x1, y1, x2, y2 = best_track.to_ltrb()
                target_center = ((x1 + x2) / 2.0, (y1 + y2) / 2.0)
                chosen_track = best_track
                print(f"Selected track {best_track.track_id} with cursor distance: {best_distance:.2f}")

            # Aim prediction + move + click hold
            if auto_aim and target_center is not None:
                now = time.time()
                if hasattr(startAutoSlicer, "last_center") and startAutoSlicer.last_center is not None and startAutoSlicer.last_time:
                    dt = now - startAutoSlicer.last_time
                    if dt > 0:
                        vx = (target_center[0] - startAutoSlicer.last_center[0]) / dt
                        vy = (target_center[1] - startAutoSlicer.last_center[1]) / dt
                        latency = 0.08
                        pred_x = target_center[0] + vx * latency
                        pred_y = target_center[1] + vy * latency
                    else:
                        pred_x, pred_y = target_center
                else:
                    pred_x, pred_y = target_center

                if point_distance((pred_x, pred_y), current_cursor_pos) < MIN_MOVEMENT_THRESHOLD:
                    print(f"Skipping move to track {chosen_track.track_id}: movement too small")
                    path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                else:
                    startAutoSlicer.last_center = target_center
                    startAutoSlicer.last_time = now
                    mapped_x = int(MON["left"] + pred_x)
                    mapped_y = int(MON["top"] + pred_y)
                    mapped_x = max(0, min(SCREEN_W - 1, mapped_x))
                    mapped_y = max(0, min(SCREEN_H - 1, mapped_y))
                    target_lock = (mapped_x, mapped_y)

                    path_safe = not class1_boxes or is_path_safe(current_cursor_pos, (pred_x, pred_y), class1_boxes)
                    if chosen_track.track_id != last_track_id:
                        if mouse_pressed:
                            win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                            mouse_pressed = False
                            time.sleep(0.005)  # Reduced delay
                        if path_safe:
                            win32api.SetCursorPos(target_lock)
                            win32api.mouse_event(0x0002, 0, 0, 0, 0)  # Press
                            mouse_pressed = True
                            startAutoSlicer.last_track_id = chosen_track.track_id
                            path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                            print(f"Mouse moved to {target_lock}, clicked")
                        else:
                            print(f"Skipping move to track {chosen_track.track_id}: unsafe path")
                            target_lock = None
                            startAutoSlicer.last_track_id = None
                    else:
                        if path_safe:
                            win32api.SetCursorPos(target_lock)
                            if not mouse_pressed:
                                win32api.mouse_event(0x0002, 0, 0, 0, 0)  # Press
                                mouse_pressed = True
                            path_to_draw = (current_cursor_pos, (pred_x, pred_y))
                            print(f"Mouse moved to {target_lock}, clicked (same track)")
                        else:
                            if mouse_pressed:
                                win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                                mouse_pressed = False
                            print(f"Releasing mouse: unsafe path for track {chosen_track.track_id}")
                            target_lock = None
                            startAutoSlicer.last_track_id = None
            else:
                if mouse_pressed:
                    win32api.mouse_event(0x0004, 0, 0, 0, 0)  # Release
                    mouse_pressed = False
                    time.sleep(0.005)
                target_lock = None
                startAutoSlicer.last_center = None
                startAutoSlicer.last_time = None
                startAutoSlicer.last_track_id = None
                print("Releasing mouse: no target or auto_aim off")

            elapsed = time.time() - t0
            fps = 1.0 / elapsed if elapsed > 0 else 0.0
            fps_deque.append(fps)
            print(f"Iteration FPS: {fps:.2f}")

            if visualize:
                frame = img.copy()
                cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                cv2.putText(frame, f"Aim: {'ON' if auto_aim else 'OFF'}", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

                # Draw only selected track and class 1 boxes
                if chosen_track is not None:
                    x1, y1, x2, y2 = map(int, chosen_track.to_ltrb())
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID {chosen_track.track_id} C0", (x1, max(0, y1-6)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                for x1, y1, x2, y2 in class1_boxes:
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

                if target_center is not None:
                    cv2.circle(frame, (int(target_center[0]), int(target_center[1])), 4, (0, 255, 255), -1)
                if target_lock is not None:
                    vis_pred_x = target_lock[0] - MON["left"]
                    vis_pred_y = target_lock[1] - MON["top"]
                    cv2.circle(frame, (int(vis_pred_x), int(vis_pred_y)), 4, (255, 0, 255), -1)
                if path_to_draw is not None:
                    start, end = path_to_draw
                    cv2.line(frame, (int(start[0]), int(start[1])), (int(end[0]), int(end[1])),
                             (255, 165, 0), 1)

                cv2.imshow("screenshot", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    cv2.destroyAllWindows()
                    break

In [11]:
startAutoSlicer(visualize=True)

Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 7.18
Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 22.73
Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 23.06
Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 21.72
Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 26.01
Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 26.26
Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 25.39
Class 0 dets: 0, Class 1 dets: 0
Confirmed class 0 tracks: 0
Releasing mouse: no target or auto_aim off
Iteration FPS: 24.14
C

### Quantization

In [35]:
!yolo export model=bestModel.pt format=engine half=True device=0 workspace=12

Ultralytics 8.3.196  Python-3.13.7 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from 'bestModel.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (18.3 MB)

ONNX: starting export with onnx 1.19.0 opset 19...
ONNX: slimming with onnxslim 0.1.68...
ONNX: export success  1.3s, saved as 'bestModel.onnx' (36.2 MB)

TensorRT: starting export with TensorRT 10.13.3.9...
[09/10/2025-13:09:52] [TRT] [W] Unable to determine GPU memory usage: In nvinfer1::getGpuMemStatsInBytes at common/extended/resources.cpp:1167
[09/10/2025-13:09:52] [TRT] [W] Unable to determine GPU memory usage: In nvinfer1::getGpuMemStatsInBytes at common/extended/resources.cpp:1167
[09/10/2025-13:09:52] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 21724, GPU 0 (MiB)


In [7]:
model = YOLO("bestModel.onnx", task="detect")

def startAutoSlicer(visualize=True):
    with mss() as sct:
        mon = sct.monitors[2]  # Monitor to capture
        fps_list = []  # To store FPS values for averaging

        while True:
            start_time = time.time()  # Start time for FPS calculation

            # Capture screenshot
            img = np.asarray(sct.grab(mon))
            img = cv2.resize(img, (640, 640))
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

            # Run YOLO inference
            results = model.predict(img, verbose=True, conf=0.7, iou=0.5)

            # Calculate FPS
            end_time = time.time()
            processing_time = end_time - start_time
            fps = 1 / processing_time if processing_time > 0 else 0
            fps_list.append(fps)

            # Optional visualization
            if visualize:
                annotated_image = results[0].plot()
                # Display FPS on the image
                cv2.putText(
                    annotated_image,
                    f"FPS: {fps:.2f}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2,
                )
                cv2.imshow("screenshot", annotated_image)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    cv2.destroyAllWindows()
                    break

        # Calculate and print average FPS
        avg_fps = np.mean(fps_list) if fps_list else 0
        print(f"Average FPS: {avg_fps:.2f}")

In [9]:
startAutoSlicer(visualize=True)


0: 640x640 (no detections), 46.6ms
Speed: 8.2ms preprocess, 46.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 50.8ms
Speed: 2.9ms preprocess, 50.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 49.4ms
Speed: 2.7ms preprocess, 49.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 47.5ms
Speed: 2.8ms preprocess, 47.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 47.5ms
Speed: 2.4ms preprocess, 47.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 45.4ms
Speed: 2.3ms preprocess, 45.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 44.8ms
Speed: 2.1ms preprocess, 44.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 37.4ms
Speed: 2.7ms preprocess, 37.4ms i

In [ ]:
# # Make process DPI aware so SetCursorPos uses the same pixel coordinates
# try:
#     ctypes.windll.user32.SetProcessDPIAware()
# except Exception:
#     pass

# # Load YOLO
# model = YOLO("best11s.pt")
# model.to("cuda")

# auto_aim = False
# target_lock = None

# # Prediction state
# last_center = None
# last_time = None

# def startAutoSlicer(visualize=True):
#     global auto_aim, target_lock, last_center, last_time

#     # Capture region = your monitor resolution (1024x768)
#     mon = {"top": 0, "left": 0, "width": 1024, "height": 768}
#     model_input = 640  # resized input used for YOLO
#     fps_deque = deque(maxlen=60)

#     # Get full screen size for clamping final cursor pos
#     screen_w = ctypes.windll.user32.GetSystemMetrics(0)
#     screen_h = ctypes.windll.user32.GetSystemMetrics(1)

#     with mss() as sct:
#         while True:
#             start_time = time.time()

#             # Capture screenshot (800x600)
#             img = np.asarray(sct.grab(mon))[:, :, :3]

#             # Resize to model input
#             img_resized = cv2.resize(img, (model_input, model_input))

#             # YOLO inference
#             results = model(img_resized, stream=False, conf=0.7, iou=0.5)

#             # Toggle with E
#             if keyboard.is_pressed("e"):
#                 auto_aim = not auto_aim
#                 print(f"Auto Aim {'ON' if auto_aim else 'OFF'}")
#                 time.sleep(0.25)

#             # Find first class 0 target (on model input coords)
#             target_center = None
#             target_id = None  # unique for switching
#             for i, box in enumerate(results[0].boxes):
#                 cls = int(box.cls[0].cpu())
#                 if cls != 0:
#                     continue

#                 x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

#                 # Check overlap with class 1 boxes
#                 overlap = False
#                 for other in results[0].boxes:
#                     other_cls = int(other.cls[0].cpu())
#                     if other_cls != 1:
#                         continue
#                     ox1, oy1, ox2, oy2 = other.xyxy[0].cpu().numpy()

#                     # IoU (Intersection over Union)
#                     inter_x1 = max(x1, ox1)
#                     inter_y1 = max(y1, oy1)
#                     inter_x2 = min(x2, ox2)
#                     inter_y2 = min(y2, oy2)
#                     inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

#                     area1 = (x2 - x1) * (y2 - y1)
#                     area2 = (ox2 - ox1) * (oy2 - oy1)
#                     union_area = area1 + area2 - inter_area

#                     iou = inter_area / union_area if union_area > 0 else 0
#                     if iou > 0.2:  # threshold, adjust if needed
#                         overlap = True
#                         break

#                 if overlap:
#                     continue  # skip this class 0 target

#                 # Found valid target
#                 cx = float((x1 + x2) / 2.0)
#                 cy = float((y1 + y2) / 2.0)
#                 target_center = (cx, cy)
#                 target_id = i
#                 break

#             # Compute scale from model input back to capture region
#             scale_x = mon["width"] / model_input   # 800 / 640 = 1.25
#             scale_y = mon["height"] / model_input  # 600 / 640 = 0.9375

#             # Prediction + mapping to actual screen coords
#             if auto_aim and (target_center is not None):
#                 now = time.time()
#                 if last_center is not None and last_time is not None:
#                     dt = now - last_time
#                     if dt > 0:
#                         vx = (target_center[0] - last_center[0]) / dt
#                         vy = (target_center[1] - last_center[1]) / dt
#                         # Predict position after model latency (~measured). adjust if needed.
#                         model_latency = 0.05
#                         pred_x = target_center[0] + vx * model_latency
#                         pred_y = target_center[1] + vy * model_latency
#                     else:
#                         pred_x, pred_y = target_center
#                 else:
#                     pred_x, pred_y = target_center

#                 last_center = target_center
#                 last_time = now

#                 # Map predicted coordinates back to capture (800x600) space,
#                 # then add capture top/left to get absolute screen coords
#                 mapped_x = int(mon["left"] + pred_x * scale_x)
#                 mapped_y = int(mon["top"] + pred_y * scale_y)

#                 # Clamp to screen bounds (safety)
#                 mapped_x = max(0, min(screen_w - 1, mapped_x))
#                 mapped_y = max(0, min(screen_h - 1, mapped_y))

#                 target_lock = (mapped_x, mapped_y)
#                 win32api.SetCursorPos(target_lock)

#                 # --- Auto left click ---
#                 if target_id != getattr(startAutoSlicer, "last_target_id", None):
#                     # switched to new target → ensure click released first
#                     win32api.mouse_event(0x0004, 0, 0, 0, 0)  # left up
#                     time.sleep(0.01)
#                 # hold down
#                 win32api.mouse_event(0x0002, 0, 0, 0, 0)  # left down
#                 startAutoSlicer.last_target_id = target_id

#             else:
#                 target_lock = None
#                 last_center = None
#                 last_time = None

#                 # --- Release left click when no target ---
#                 if getattr(startAutoSlicer, "last_target_id", None) is not None:
#                     win32api.mouse_event(0x0004, 0, 0, 0, 0)  # left up
#                     startAutoSlicer.last_target_id = None

#             # FPS
#             fps = 1.0 / (time.time() - start_time) if (time.time() - start_time) > 0 else 0.0
#             fps_deque.append(fps)

#             if visualize:
#                 # Show original capture (800x600) so visual coords match cursor coords
#                 frame = img.copy()  # already 800x600
#                 cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
#                             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#                 cv2.putText(frame, f"Aim: {'ON' if auto_aim else 'OFF'}", (10, 70),
#                             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

#                 # Draw detected center (mapped to 800x600) and predicted cursor
#                 if target_center is not None:
#                     vis_cx = int(target_center[0] * scale_x)
#                     vis_cy = int(target_center[1] * scale_y)
#                     cv2.circle(frame, (vis_cx, vis_cy), 6, (0, 255, 255), -1)  # detected center (yellow)

#                 if target_lock is not None:
#                     # target_lock is in screen coords, subtract mon left/top to draw on frame
#                     vis_pred_x = int(target_lock[0] - mon["left"])
#                     vis_pred_y = int(target_lock[1] - mon["top"])
#                     cv2.circle(frame, (vis_pred_x, vis_pred_y), 6, (0, 0, 255), -1)  # predicted cursor (red)

#                 cv2.imshow("screenshot", frame)
#                 if cv2.waitKey(1) & 0xFF == ord("q"):
#                     cv2.destroyAllWindows()
#                     break